In [1]:
# from telegram import (
#     Poll,
#     ForceReply,
#     ParseMode,
#     KeyboardButton,
#     KeyboardButtonPollType,
#     ReplyKeyboardMarkup,
#     ReplyKeyboardRemove,
#     Update,
# )

# from telegram.ext import (
#     Updater,
#     CommandHandler,
#     PollAnswerHandler,
#     PollHandler,
#     MessageHandler,
#     Filters,
#     CallbackContext,
# )


def start(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, _: CallbackContext) -> None:
    """Echo the user message."""
    print(update.message.text)
    update.message.reply_text(update.message.text)
    
    
def poll(update: Update, context: CallbackContext) -> None:
    """Sends a predefined poll"""
    print(update.message.text)
    x, y = update.message.text.split(' ')[1:3]
    
    questions = ["Boo", "Foo", "Doo", "Goo"]
    if len(update.message.text.split(' ')) > 3:
        questions.extend(update.message.text.split(' ')[2:])
    message = context.bot.send_poll(
        update.effective_chat.id,
        f"Отдать {int(x)/len(questions):,} рублей с каждого {y} Всего {int(x):,} рублей",
        questions,
        is_anonymous=False,
        allows_multiple_answers=True,
    )
    # Save some info about the poll the bot_data for later use in receive_poll_answer
    payload = {
        message.poll.id: {
            "questions": questions,
            "message_id": message.message_id,
            "chat_id": update.effective_chat.id,
            "answers": 0,
        }
    }
    context.bot_data.update(payload)
    
    
def receive_poll_answer(update: Update, context: CallbackContext) -> None:
    """Summarize a users poll vote"""
    answer = update.poll_answer
    poll_id = answer.poll_id
    try:
        questions = context.bot_data[poll_id]["questions"]
    # this means this poll answer update is from an old poll, we can't do our answering then
    except KeyError:
        return
    selected_options = answer.option_ids
    answer_string = ""
    for question_id in selected_options:
        if question_id != selected_options[-1]:
            answer_string += questions[question_id] + " и "
        else:
            answer_string += questions[question_id]
    context.bot.send_message(
        context.bot_data[poll_id]["chat_id"],
        f"{update.effective_user.mention_html()} отдал за {answer_string}!",
        parse_mode=ParseMode.HTML,
    )
    context.bot_data[poll_id]["answers"] += 1
    # Close poll after three participants voted
    if context.bot_data[poll_id]["answers"] == 3:
        context.bot.stop_poll(
            context.bot_data[poll_id]["chat_id"], context.bot_data[poll_id]["message_id"]
        )


def main() -> None:
    # Create the Updater and pass it your bot's token.
    updater = Updater("token")
    dispatcher = updater.dispatcher
    dispatcher.add_handler(CommandHandler('start', start))
    dispatcher.add_handler(CommandHandler('split_money', poll))
    dispatcher.add_handler(PollAnswerHandler(receive_poll_answer))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))
#     dispatcher.add_handler(PollHandler(receive_quiz_answer))
#     dispatcher.add_handler(CommandHandler('preview', preview))
#     dispatcher.add_handler(MessageHandler(Filters.poll, receive_poll))
#     dispatcher.add_handler(CommandHandler('help', help_handler))

    # Start the Bot
    updater.start_polling()

    # Run the bot until the user presses Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT
    updater.idle()


if __name__ == '__main__':
    main()

NameError: name 'Update' is not defined

In [3]:
import cv2
# Read the image
# img = cv2.imread('0001.jpg',0)
# Simple thresholding
# ret, thresh1 = cv2.threshold(img,210,255,cv2.THRESH_BINARY)

# cv2.imshow('gray', thresh1 )
# cv2.waitKey(0) 

In [35]:
import pytesseract
from pytesseract import Output

img = cv2.imread('0010.jpg')
ret, thresh1 = cv2.threshold(img,210,255,cv2.THRESH_BINARY)
d = pytesseract.image_to_string(thresh1, lang='rus')



In [36]:
print(d)

******************************************

"'*'-Г “'

тезт .
**********************ж***********жж******
010101540. в
1 Х 120.00 = 120.00
Картофель с маслом
010101020. 3
1 Х 61.00 = 61.00
Нап. крабовое мясо
0101010533. :
1 Х 68.00 = 68.00
Нап.ветчина с прянои ад
010101007. в
1 47.00 = 47.00
Чесночный хлеб
ИЦИЙ:
ШШ: 296.00
ОПЛАТА

Саго ХХ М ХХХХХХХХХХХХХХХХХХ: 296.00
СПНД # 29814 Транзакция # 00000

Отдел # 01 01

ККТ # ТГФР18—00016790 Касса # 001
ЁЁССОВЫЙ чек

9287440300630943

РН ККТ 0002472314002805
; ИНН 500910091123
` ФД 118346

18.05.21 21:35

ФП 3688043572

СМЕНА 529

ЧЕК 269

ПРИХОД

ИТОГ

296.00
ИП КРАСАНОВА НАТАЛЬЯ АНАТОЛЬЕВНА
г. Москва, площадь Киевского вокзала, д. 2, '
ТРЦ' 'Европейский"
МЕСТО РАСЧЕТОВ Кафе' 'Крошка-Картошка"
КАССИР Кассир: Хабова А

СНО Патент '

САЙТ ФНС лил.паТоо.ги
БЕЗНАЛИЧНЫМИ 296.00
СУММА БЕЗ НДС '296.00

 

‘“ Спасибо за заказ!
Номер заказа: 1- 223631
Длительность обслуживания: 00:47



In [86]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image, 5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated


In [87]:
import re
import pytesseract
from pytesseract import Output

img = cv2.imread('0010.jpg')
# ret, thresh1 = cv2.threshold(img,210,255,cv2.THRESH_BINARY)
d = pytesseract.image_to_string(remove_noise(img), lang='rus')

d_without_n = [t for t in d.split('\n') if len(t) != 0] 
d_without_n

if [t.split('=') for t in d_without_n] != d_without_n:
    d_without_n = [t.split('=') for t in d_without_n]
    d_without_n = [j for t in d_without_n for j in t]
d_without_n

dish = [t for t in d_without_n if (not re.match(r'\d+\.\d+', t)) and (len(t)>7)]
cost = [t for t in d_without_n if re.match(r'\d+\.\d+', t)]
cost, dish
# pd.DataFrame(dish, cost)

(['18.05.21 21:35', '296.00'],
 ['******************************************',
  '********************ж*********************',
  '010101540. в',
  '1 Х 120.00 ',
  'Картофель с маслом',
  '010101020. #',
  '1 Х 61.00 ',
  'Нап. коабовое мясо',
  '0101010533. #',
  '1 Х 68.00 ',
  'Нап.ветчина с полной ад',
  '010101007. в',
  '1 Х 47.00 ',
  'Чесночный хлеб',
  'ПОЗИЦИИ: 4',
  'ИТОГО: 296 . 00',
  'Саго ХХ М ХХХХХХХХХХХХХХХХХХ: 296.00',
  'СПНД # 29814 Транзакция # 00000',
  'Отдел # 01 0001',
  'ККТ # ТГФР18—0001б790 Касса # 001',
  'Ёдссовый чек',
  '9287440300630943',
  'РН ККТ 0002472314002805',
  'ИНН 500910091123',
  'ФД 118346',
  'ФП 3688043572',
  'СМЕНА 529',
  'ИП КРАСАНОВА НАТАЛЬЯ АНАТОЛЬЕВНА',
  'г. Москва, площадь Киевского вокзала, д.2,',
  'ТРЦ ”Европейский"',
  'МЕСТО РАСЧЕТОВ Кафе ”Крошка-Картошка"',
  'КАССИР Кассир: Хабова А',
  'СНО Патент :',
  'САЙТ ФНС ммм.па109.ги',
  'БЕЗНАЛИЧНЫМИ 296.00',
  'СУММА БЕЗ НДС 296.00',
  'Спасибо за заказ!',
  'Номер заказа: 1-223

In [422]:
REGEX_PATTERNS_NUMBERS = ['\d+\.00','\d+\.ОО','\d+\.OO','\d+\.оо', '\d+\.oo',
                          '\d+\. 00', '\d+\. ОО','\d+\. OO','\d+\. оо','\d+\. oo',
                          '\d+ \. 00', '\d+ \. ОО','\d+ \. OO','\d+ \. оо','\d+ \. oo']

REGEX_PATTERNS_NUMBERS_EASIER = ['\d+\D00','\d+\DОО','\d+\DOO','\d+\Dоо', '\d+\Doo',
                          '\d+\D 00', '\d+\D ОО','\d+\D OO','\d+\D оо','\d+\D oo',
                         '\d+ 00', '\d+ ОО','\d+ OO','\d+ оо','\d+ oo',
                          '\d+ \D 00', '\d+ \D ОО','\d+ \D OO','\d+ \D оо','\d+ \D oo']

img_list = ['0001.jpg','0002.jpg','0003.jpg','0004.jpg','0005.jpg','0006.jpg','0007.jpg','0008.jpg','0009.jpg','0010.jpg','0011.jpg','0012.jpg','0013.jpg','0014.jpg']

def price_pattern_in_text(text, easy = False):
    if easy == True:
        patterns = REGEX_PATTERNS_NUMBERS  + REGEX_PATTERNS_NUMBERS_EASIER
    else:
        patterns = REGEX_PATTERNS_NUMBERS
    matches_list = []
    for regex in patterns:
        matches = re.findall(regex, text)
        matches_list += matches
    return matches_list != []

def price_pattern_one_line(text, easy = False):
    if easy == True:
        patterns = REGEX_PATTERNS_NUMBERS + REGEX_PATTERNS_NUMBERS_EASIER
    else:
        patterns = REGEX_PATTERNS_NUMBERS
    matches_list = []
    for regex in patterns:
        matches = re.findall(regex, text)
        matches_list += matches
    return list(set(matches_list))
    
def get_all_text(img_name):
    img = cv2.imread(img_name)
    d = pytesseract.image_to_string(remove_noise(img), lang='rus')
    return d

def clean_lines(d):
    d_list = d.split('\n')
    d_match_list =[]
    for x in d_list:
        if price_pattern_in_text(x):
            d_match_list = d_match_list + [x]
    return d_match_list 

# for img_name in img_list[0:2]:
#     d = get_all_text(img_name)
#     print(d)
#     for x in clean_lines(d):
#         print(x)
    

In [236]:
def get_right_column(img_name, round_factor = 20):
    """
    Функция по сути ищет крайнюю правую колонку коробочек которую нашел tesseract
    """
    img = cv2.imread(img_name)
    d = pytesseract.image_to_data(remove_noise(img), output_type=Output.DICT, lang='rus')
    df = pd.DataFrame(d)
    df = df[df['text']!='']
    df['top_right'] = df['left'] + df['width']
    pd.set_option('display.max_rows', None)
    df['left_round'] = round(df['left']/round_factor)*round_factor
    df['right_round'] = round(df['top_right']/round_factor)*round_factor
    #df_left = df.groupby('left_round').count().sort_values('level', ascending=False )
    #df_right = df.groupby('right_round').count().sort_values('level', ascending=False)
    max_right = max(df['right_round'])
    right_column = df[df['right_round']==max_right]
    return right_column

In [423]:
def get_table_regex_matched(img_name, round_factor):
    """
    Находим в строчках, которые доходят до конца, совпадения с регексом цены - сначала строгого,
    потом менее строгого для строчек которые по расположению между совпадениями. Не факт что лучше обычных регексов 
    в рамках поиска цифр но должно помочь когда будем искать названия.
    """
    img = cv2.imread(img_name)
    d = pytesseract.image_to_data(remove_noise(img), output_type=Output.DICT, lang='rus')
    test_df_full = pd.DataFrame(d)
    test_df_right = get_right_column(img_name, round_factor)
    cumulative_line_table = test_df_full.groupby(
        ['block_num','line_num']
    ).count().reset_index().reset_index()[['index','block_num','line_num']].rename(
        columns = {'index':'cumulative_line'}
    )
    test_df_joined = test_df_full.join(
    test_df_right[['block_num','line_num']].set_index(['block_num','line_num']),
        on=['block_num','line_num'], 
        how = 'inner'
    ).join(
    cumulative_line_table.set_index(['block_num','line_num']),
        on=['block_num','line_num'], 
        how = 'inner'
    )
    test_df_joined['full_text'] = test_df_joined.groupby(['block_num','line_num'])['text'].transform(lambda x: ' '.join(x))
    test_df_joined['price_match_bool'] = test_df_joined['full_text'].apply(lambda x: price_pattern_in_text(x))
    test_df_joined['price_match_number'] = test_df_joined['full_text'].apply(lambda x: price_pattern_one_line(x))
    max_true = test_df_joined[test_df_joined['price_match_bool']]['cumulative_line'].max()
    min_true = test_df_joined[test_df_joined['price_match_bool']]['cumulative_line'].min()
    test_df_joined['price_match_bool_easy'] = test_df_joined.apply(
        lambda x: easy_price_regex_bool(max_true,min_true,x['cumulative_line'], x['price_match_bool'], x['full_text']), axis = 1
    )
    test_df_joined['price_match_number'] = test_df_joined.apply(
        lambda x: easy_price_regex_one_line(max_true,min_true,x['cumulative_line'], x['price_match_number'], x['full_text']), axis = 1
    )
    return test_df_joined

In [542]:
from itertools import combinations    
def combs(lst):
    """
    Все комбинации найденных цифр
    """
    return (c for k in range(len(lst), 0, -1) for c in combinations(lst, k))

def possible_combos(lst):
    """
    Ищет комбинацию из найденных цифр похожих на цену, совпадающую с самой большой из них
    Если не находит, переходит к следующей по величине.
    Нужно оптимизировать и сделать поправку на количество
    """
    lst_sorted = lst.copy()
    lst_sorted.sort(reverse=True)
    for i in lst_sorted:
        lst = [x for x in lst if x!= i]
        lst = [x for x in lst if x!= 0]
        combs_lst =combs(lst)
        possible_combs = []
        target = i
        for c in combs_lst:
            if sum(c) == target:
                possible_combs.append(c)
        if possible_combs != []:
            break
    return possible_combs

In [484]:
get_right_column('0001.jpg', 30)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,top_right,left_round,right_round
5,5,1,1,1,1,2,1135,103,1280,87,63,_—…—__________…13:44_…___…,2415,1140.0,2400.0
51,5,1,8,1,1,4,2318,1076,91,85,97,00,2409,2310.0,2400.0
58,5,1,8,1,2,6,2318,1217,91,84,98,00,2409,2310.0,2400.0
65,5,1,8,1,3,6,2317,1356,92,84,96,00,2409,2310.0,2400.0
73,5,1,8,1,4,7,2317,1495,92,85,97,00,2409,2310.0,2400.0
80,5,1,8,1,5,6,2317,1635,92,83,95,00,2409,2310.0,2400.0
89,5,1,8,1,6,8,2317,1774,92,84,96,00,2409,2310.0,2400.0
96,5,1,8,1,7,6,2317,1913,92,84,95,00,2409,2310.0,2400.0
104,5,1,8,1,8,7,2317,2054,92,85,94,00,2409,2310.0,2400.0
111,5,1,8,1,9,6,2318,2195,91,85,95,00,2409,2310.0,2400.0


In [524]:
regex_matched_df = get_table_regex_matched('0001.jpg', 30)
number_lines = regex_matched_df[regex_matched_df['price_match_bool_easy']][['full_text','price_match_number','cumulative_line']]#.drop_duplicates()
number_lines['price_match_number'] = number_lines['price_match_number'].apply(lambda x: x[0])
possible_items = number_lines.drop_duplicates()
possible_items

,full_text,price_match_number,cumulative_line
47,1 420.00 420 00,420.00,18
52,Домашний салат 1 240.00 240 00,240.00,19
59,Картофель Пай 1 380.00 380 00,380.00,20
66,"Ледяная Жемчужина 0,5л 1 120.00 120 00",120.00,21
74,Мохито б/а 1 200.00 200 00,200.00,22
81,Моцарелла с помидорами 1 480 00 480 00,480 00,23
90,Пельмени домашние 1 240.00 240 00,240.00,24
97,Сладкий цветок 1 220 00 220 00,220 00,25
105,Спелые ягоды 1 220.00 220 00,220.00,26
112,Сырные палочки 1 280.00 280 00,280.00,27


In [525]:
# the longest one is perfect
test_list = list(possible_items['price_match_number'].apply(lambda x: float(x.replace(' ','.'))))
possible_combos(test_list)

[(420.0, 240.0, 380.0, 120.0, 200.0, 480.0, 240.0, 220.0, 220.0, 280.0, 320.0),
 (420.0, 240.0, 380.0, 120.0, 200.0, 240.0, 220.0, 220.0, 320.0, 760.0),
 (420.0, 240.0, 380.0, 120.0, 200.0, 480.0, 240.0, 280.0, 760.0),
 (420.0, 240.0, 380.0, 120.0, 480.0, 220.0, 220.0, 280.0, 760.0),
 (420.0, 380.0, 120.0, 200.0, 480.0, 220.0, 220.0, 320.0, 760.0),
 (420.0, 380.0, 120.0, 480.0, 240.0, 220.0, 220.0, 280.0, 760.0),
 (240.0, 380.0, 200.0, 480.0, 240.0, 220.0, 280.0, 320.0, 760.0),
 (240.0, 380.0, 200.0, 480.0, 240.0, 220.0, 280.0, 320.0, 760.0),
 (420.0, 240.0, 380.0, 480.0, 240.0, 280.0, 320.0, 760.0),
 (120.0, 200.0, 220.0, 220.0, 2360.0),
 (420.0, 120.0, 220.0, 2360.0),
 (420.0, 120.0, 220.0, 2360.0),
 (240.0, 200.0, 320.0, 2360.0),
 (240.0, 240.0, 280.0, 2360.0),
 (200.0, 240.0, 320.0, 2360.0),
 (220.0, 220.0, 320.0, 2360.0),
 (480.0, 280.0, 2360.0),
 (760.0, 2360.0)]

In [426]:
get_right_column('0002.jpg',140)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,top_right,left_round,right_round
23,5,1,1,2,3,3,742,258,308,44,60,ООО.НС0000СО$.ВО,1050,700.0,1120.0
32,5,1,2,1,1,1,26,366,1103,29,57,___—___.,1129,0.0,1120.0
38,5,1,2,1,2,5,939,413,189,31,49,77`0°д4`°0,1128,980.0,1120.0
54,5,1,2,3,1,4,1043,528,96,31,73,50.00,1139,980.0,1120.0
60,5,1,3,1,1,3,1046,568,96,39,80,50.00,1142,980.0,1120.0
66,5,1,3,1,2,5,1049,613,95,36,66,51-00,1144,980.0,1120.0
70,5,1,3,1,3,3,1052,658,93,38,80,51.00,1145,1120.0,1120.0
78,5,1,3,1,4,7,1054,709,94,35,85,75.00,1148,1120.0,1120.0
82,5,1,3,1,5,3,1055,757,96,35,82,75.00,1151,1120.0,1120.0
89,5,1,3,1,6,6,1057,805,96,33,77,95.00,1153,1120.0,1120.0


In [552]:
#Some weird matches, but would be kind of okay with a sum check
regex_matched_df = get_table_regex_matched('0002.jpg', 140)
number_lines = regex_matched_df[regex_matched_df['price_match_bool_easy']][['full_text','price_match_number','cumulative_line','block_num','line_num']]#.drop_duplicates()
number_lines['price_match_number'] = number_lines['price_match_number'].apply(lambda x: x[0])
possible_items = number_lines.drop_duplicates()
possible_items

,full_text,price_match_number,cumulative_line,block_num,line_num
31,___—___. ___—___. КЯССОВНЙ КЯССОВНЙ ЧЕК ЧЕ...,50.00,7,2,1
57,ндс 202 50.00,50.00,11,3,1
61,кнетооель-оеи ннл. \ Х 51-00,51-00,12,3,2
67,НОС 208 51.00,51.00,13,3,3
71,вод нквр минерале нет 1 х 75.00,75.00,14,3,4
79,НВС 203 75.00,75.00,15,3,5
83,палочки геик чиз 1 х 95.00,95.00,16,3,6
90,_ __ __ _ НЛС 20% 95.00,95.00,17,3,7
98,КОКО-КОЛО КОКО-КОЛО 5005000 5005000 1 1 х х ...,79 00,18,3,8
105,‘ _ ндс_20%_ 79.00,79.00,19,3,9


In [553]:
# # this one takes too long to work. fix the algorithm
# test_list = list(possible_items['price_match_number'].apply(lambda x: float(x.replace(' ','.').replace('-','.').replace('„','.').replace('..','.'))))
# possible_combos(test_list)

KeyboardInterrupt: 

In [428]:
get_right_column('0003.jpg',20)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,top_right,left_round,right_round
44,5,1,1,1,9,10,1975,1194,370,77,90,.=220.00,2345,1980.0,2340.0
52,5,1,1,1,10,7,2015,1286,329,75,89,=190.00,2344,2020.0,2340.0
64,5,1,1,1,11,11,2016,1378,328,78,85,=220.00,2344,2020.0,2340.0
75,5,1,1,1,13,6,1877,1566,466,77,89,1..=220.00,2343,1880.0,2340.0
83,5,1,1,1,14,7,2015,1658,328,78,90,=220.00,2343,2020.0,2340.0
90,5,1,2,1,1,4,1696,2014,638,157,89,1070.00,2334,1700.0,2340.0


In [526]:
#perfect but text isn't too good
regex_matched_df = get_table_regex_matched('0003.jpg', 20)
number_lines = regex_matched_df[regex_matched_df['price_match_bool_easy']][['full_text','price_match_number','cumulative_line']]#.drop_duplicates()
number_lines['price_match_number'] = number_lines['price_match_number'].apply(lambda x: x[0])
possible_items = number_lines.drop_duplicates()
possible_items

,full_text,price_match_number,cumulative_line
34,1. домдшнии чди мятндя ГРУША. .220. 00 х 1. ....,220.00,10
45,2. КДР. ‚190. 00х 1 ........................ ...,190. 00,11
53,3. ТРИ шдрикд С хСИРОПОМ. .220 00 х 1 ..........,220.00,12
69,4.НдП0пЕ0Н с здвцгн. КРЕМОМ..220.00 х 1..=220.00,220.00,14
76,5.МЕДОВИК С ЧЕРНОСЛИВОМ..220.00 х 1 ...... =2...,220.00,15
86,ИТОГО К ОПЛдТЕі. 1070.00,1070.00,17


In [527]:
# the longest one is perfect
test_list = list(possible_items['price_match_number'].apply(lambda x: float(x.replace(' ','.').replace('-','.').replace('„','.').replace('..','.'))))
possible_combos(test_list)

[(220.0, 190.0, 220.0, 220.0, 220.0)]

In [430]:
#This one does not work at all - need to manual search the dots
get_right_column('0004.jpg',20)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,top_right,left_round,right_round
5,5,1,1,1,1,2,1496,0,1804,260,19,дн,3300,1500.0,3300.0


In [514]:
#this one isn't okay
regex_matched_df = get_table_regex_matched('0004.jpg', 20)
number_lines = regex_matched_df[regex_matched_df['price_match_bool_easy']][['full_text','price_match_number','cumulative_line']]#.drop_duplicates()
number_lines['price_match_number'] = number_lines['price_match_number'].apply(lambda x: x[0])
possible_items = number_lines.drop_duplicates()
possible_items

,full_text,price_match_number,cumulative_line


In [515]:
# still not working
test_list = list(possible_items['price_match_number'].apply(lambda x: float(x.replace(' ','.').replace('-','.').replace('„','.').replace('..','.'))))
possible_combos(test_list)

UnboundLocalError: local variable 'possible_combs' referenced before assignment

In [432]:
get_right_column('0005.jpg',20)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,top_right,left_round,right_round
4,5,1,1,1,1,1,13,252,1088,24,51,йііііііііійіійіійіі***іійі***90909*і*'**ііііійіі,1101,20.0,1100.0
19,5,1,3,1,2,1,15,451,1086,24,52,{***іііййііййі*і**і**ййііідійі**й**йіі*іі**іі*йі,1101,20.0,1100.0
27,5,1,4,1,1,5,990,496,110,40,77,19:11,1100,1000.0,1100.0
34,5,1,4,1,2,6,1036,545,59,41,85,161,1095,1040.0,1100.0
41,5,1,4,2,1,2,669,644,430,41,76,ФН=9282440300760803,1099,660.0,1100.0
46,5,1,4,2,2,4,987,692,112,40,66,#2730,1099,980.0,1100.0
50,5,1,4,2,3,3,827,742,271,40,64,иич.па|оя.ги,1098,820.0,1100.0
77,5,1,6,1,3,2,919,984,176,41,85,125.00_Г,1095,920.0,1100.0
81,5,1,6,1,4,3,1008,1035,87,40,87,0.00,1095,1000.0,1100.0
87,5,1,6,1,6,2,940,1130,154,41,83,75.00-Г,1094,940.0,1100.0


In [528]:
#the prices are okay but need to look up text. Also the triple equal sign turned out weird.
regex_matched_df = get_table_regex_matched('0005.jpg', 20)
number_lines = regex_matched_df[regex_matched_df['price_match_bool_easy']][['full_text','price_match_number','cumulative_line']]#.drop_duplicates()
number_lines['price_match_number'] = number_lines['price_match_number'].apply(lambda x: x[0])
possible_items = number_lines.drop_duplicates()
possible_items

,full_text,price_match_number,cumulative_line
75,= 125.00_Г,125.00,19
78,БЕЗ НАС 0.00,0.00,20
85,= 75.00-Г,75.00,22
88,БЕЗ НАС 0.00,0.00,23
96,= 69.00_Г,69.00,25
99,БЕЗ НАС 0.00,0.00,26
107,: 107.00_Г,107.00,28
110,БЕЗ НАС 0.00,0.00,29
117,= 85.00_Г,85.00,31
120,БЕЗ ндс 0.00,0.00,32


In [529]:
# this one is not working because the triple equal sign read weird
test_list = list(possible_items['price_match_number'].apply(lambda x: float(x.replace(' ','.').replace('-','.').replace('„','.').replace('..','.'))))
possible_combos(test_list)

ValueError: list.remove(x): x not in list

In [ ]:
get_right_column('0006.jpg',20)

In [530]:
#Same thing as above, would work with an algorithm though
regex_matched_df = get_table_regex_matched('0006.jpg', 20)
number_lines = regex_matched_df[regex_matched_df['price_match_bool_easy']][['full_text','price_match_number','cumulative_line']]#.drop_duplicates()
number_lines['price_match_number'] = number_lines['price_match_number'].apply(lambda x: x[0])
possible_items = number_lines.drop_duplicates()
possible_items

,full_text,price_match_number,cumulative_line
72,= 135.00_Г,135.00,18
75,БЕЗ нпс 0.00,0.00,19
82,: 59.00_Г,59.00,21
85,БЕЗ ндс 0.00,0.00,22
92,= 59.00_Г,59.00,24
95,БЕЗ нпс 0.00,0.00,25
102,= 85.00_Г,85.00,27
105,БЕЗ НДС 0.00,0.00,28
109,=338.00,338.00,29
111,"""ЁЁЕКТРОННЫНИ 5338.00",5338.00,30


In [543]:
# perfect
test_list = list(possible_items['price_match_number'].apply(lambda x: float(x.replace(' ','.').replace('-','.').replace('„','.').replace('..','.'))))
possible_combos(test_list)

[(135.0, 59.0, 59.0, 85.0)]

In [ ]:
get_right_column('0007.jpg',20)

In [544]:
#this one isn't working for some reason
regex_matched_df = get_table_regex_matched('0007.jpg', 20)
number_lines = regex_matched_df[regex_matched_df['price_match_bool_easy']][['full_text','price_match_number','cumulative_line']]#.drop_duplicates()
number_lines['price_match_number'] = number_lines['price_match_number'].apply(lambda x: x[0])
possible_items = number_lines.drop_duplicates()
possible_items

,full_text,price_match_number,cumulative_line
54,КОКО-КОЛО КОКО-КОЛО КОКО-КОЛО КОКО-КО...,79.00,14


In [545]:
# not working
test_list = list(possible_items['price_match_number'].apply(lambda x: float(x.replace(' ','.').replace('-','.').replace('„','.').replace('..','.'))))
possible_combos(test_list)

[]

In [ ]:
# This one is weird
get_right_column('0008.jpg',20)

In [546]:
#the tripple equal sign is weird but kind of okay otherwise. See that 2336.13 is not picked up!
regex_matched_df = get_table_regex_matched('0008.jpg', 20)
number_lines = regex_matched_df[regex_matched_df['price_match_bool_easy']][['full_text','price_match_number','cumulative_line']]#.drop_duplicates()
number_lines['price_match_number'] = number_lines['price_match_number'].apply(lambda x: x[0])
possible_items = number_lines.drop_duplicates()
possible_items

,full_text,price_match_number,cumulative_line
147,1 Х 199.00,199.00,37
151,5199. 00- НДС 207,5199. 00,38
172,1Х Х90. 00,90. 00,44
176,8 590. 00_ НДС 102,590. 00,45
198,1 Х 90. 00,90. 00,51
203,=90. 00- НПС 107,90. 00,52


In [547]:
# not working because of the triple equal sign
test_list = list(possible_items['price_match_number'].apply(lambda x: float(x.replace(' ','.').replace('-','.').replace('„','.').replace('..','.'))))
possible_combos(test_list)

[]

In [ ]:
get_right_column('0009.jpg',100)

In [548]:
#should definetly at least pick up "=790.00" see what is up with that
regex_matched_df = get_table_regex_matched('0009.jpg', 100)
number_lines = regex_matched_df[regex_matched_df['price_match_bool_easy']][['full_text','price_match_number','cumulative_line']]#.drop_duplicates()
number_lines['price_match_number'] = number_lines['price_match_number'].apply(lambda x: x[0])
possible_items = number_lines.drop_duplicates()
possible_items

,full_text,price_match_number,cumulative_line
47,Артикул Артикул Артикул 52030402 52030402 5...,1690.00,10
206,БЕЗНдЛИЧНЫМИ =4170. 00,4170. 00,26


In [549]:
# see comments above - not picking up things it really should
test_list = list(possible_items['price_match_number'].apply(lambda x: float(x.replace(' ','.').replace('-','.').replace('„','.').replace('..','.'))))
possible_combos(test_list)

[]

In [ ]:
get_right_column('0010.jpg',70)

In [550]:
#almost okay! fix for a non-digit character before the zeros
regex_matched_df = get_table_regex_matched('0010.jpg', 70)
number_lines = regex_matched_df[regex_matched_df['price_match_bool_easy']][['full_text','price_match_number','cumulative_line']]#.drop_duplicates()
number_lines['price_match_number'] = number_lines['price_match_number'].apply(lambda x: x[0])
possible_items = number_lines.drop_duplicates()
possible_items

,full_text,price_match_number,cumulative_line
14,1 Х 120.00 = 120.00,120.00,7
27,1 Х 61.00 = 61.00,61.00,10
41,Х 68.00 = 68.00,68.00,13
55,1 Х 47.00 = 47.00,47.00,16
76,Саго ХХ И ХХХХХХХХХХХХХХХХХХ: 296.00 Отдел #...,296.00,22
110,РН РН ККТ ККТ 0002472314002805 0002472314002...,0002472314002805 00,26
142,296.00 296.00 296.00 296.00 ИП ИП ИП И...,296.00,31
183,БЕЗНАЛИЧНЫМИ 296.00,296.00,35
186,СУММА БЕЗ НДС 296.00,296.00,36


In [551]:
# this is perfect
test_list = list(possible_items['price_match_number'].apply(lambda x: float(x.replace(' ','.').replace('-','.').replace('„','.').replace('..','.'))))
possible_combos(test_list)

[(120.0, 61.0, 68.0, 47.0)]

In [444]:
TEST_DICT = {
    '0001.img':{
        'item1':{
            'price': 420
            'count': 1
        }, 
        'item2':{
            'price': 240
            'count': 1
        },
        'item3':{
            'price': 380
            'count': 1
        },
        'item4':{
            'price': 120
            'count': 1
        },
        'item5':{
            'price': 200
            'count': 1
        },
        'item6':{
            'price': 480
            'count': 1
        },
        'item7':{
            'price': 240
            'count': 1
        },
        'item8':{
            'price': 220
            'count': 1
        },
        'item9':{
            'price': 220
            'count': 1
        },
        'item10':{
            'price': 280
            'count': 1
        },
        'item11':{
            'price': 320
            'count': 1
        }
    },
    '0002.img':{
        'item1':{
            'price': 50
            'count': 1
        }, 
        'item2':{
            'price': 51
            'count': 1
        },
        'item3':{
            'price': 75
            'count': 1
        },
        'item4':{
            'price': 95
            'count': 1
        },
        'item5':{
            'price': 79
            'count': 1
        },
        'item6':{
            'price': 131
            'count': 1
        },
        'item7':{
            'price': 72
            'count': 1
        },
        'item8':{
            'price': 30
            'count': 1
        },
        'item9':{
            'price': 70
            'count': 1
        },
        'item10':{
            'price': 203
            'count': 1
        },
        'item11':{
            'price': 72
            'count': 1
        },
        'item12':{
            'price': 30
            'count': 1
        },
        'item13':{
            'price': 70
            'count': 1
        },
        'item14':{
            'price': 244
            'count': 1
        },
        'item15':{
            'price': 30
            'count': 1
        },
        'item16':{
            'price': 30
            'count': 1
        },
        'item17':{
            'price': 30
            'count': 1
        }
    },
    '0003.img':{
        'item1':{
            'price': 220
            'count': 1
        }, 
        'item2':{
            'price': 190
            'count': 1
        },
        'item3':{
            'price': 220
            'count': 1
        },
        'item4':{
            'price': 220
            'count': 1
        },
        'item5':{
            'price': 220
            'count': 1
        }
    },
    '0004.img':{
        'item1':{
            'price': 125
            'count': 1
        }
    },
    '0005.img':{
        'item1':{
            'price': 125
            'count': 1
        }, 
        'item2':{
            'price': 75
            'count': 1
        },
        'item3':{
            'price': 69
            'count': 1
        },
        'item4':{
            'price': 107
            'count': 1
        },
        'item5':{
            'price': 85
            'count': 1
        }
    }
}

SyntaxError: invalid syntax (<ipython-input-444-bae334b4646f>, line 5)